# Performance of Outlier Detection on Smartwatch Data in Single and Multiple Person Environments
## An analysis and comparison of different outlier and anomaly detection methods when applied to heart rate and step count data from consumer-grade wearables
Luuk Wubben

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import scipy as sc
import sklearn as sk
import matplotlib.pyplot as plt
import librosa as lib
import random
import time
from tqdm.notebook import tqdm
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import itertools
from typing import Iterable
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from kneebow.rotor import Rotor

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

#in case you don't have some of the above libraries, comment them out, then use the below lines to download them first.
# import sys
# !conda install --yes --prefix {sys.prefix} umap-learn

In [ ]:
#GENERAL VARIABLES. ADVISE TO OVERRIDE PER ALGORITHM PER ENVIRONMENT BELOW.
#Data Preprocessing Variables
data_sel = {
    'hr': True,
    'norm_hr': True,
    'steps': True,
    'norm_steps': True
}
features = {
    'count': False,
    'mean': True,
    'std': True,
    'min': True,
    'percentiles': [0.25, 0.5, 0.75],
    'max': True,
    'n_th_step': True,
    'mfcc': True
}
n_th_step_moment = 6
using_feat = [x for x, y in features.items() if (type(y) == bool and y) or (type(y) == list and len(y) > 0)]
window = {
    'size': 3600,     #in seconds
    'stride': 900       #in seconds
}
subjects_path = './path/to/subject/data'
subject_directory_files = os.listdir(subjects_path)
subjects = []
#Append '.pkl' if not included. Pickled files only.
# Data should be pickled Pandas DataFrames per subject consisting of timeseries data with columns 'time',
# 'hr' and 'steps'
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]
combine_subject_dataframes = True

#the variable below dictates how many data points from different subjects should be mixed in, as a value between 0 and 1.
#e.g. a value of 0.05 will take |subject_1_data|*0.05 data entries from a random person in the subjects list and insert
#them into the dataframe of subject_1. 0 is no mixing, 1 is adding a whole extra dataframe
mixing = 0.1

#Select for single person and multiple person whether outliers should be within subject/group,
#or between subject/group and a set of outliers from another random subject.
outlier_selection = {
    'within_subject': True,
    'between_subjects': False,
    'within_group': False,
    'between_group_and_individual': True
}

#Validate outlier selection
assert not (outlier_selection['within_subject'] and outlier_selection['between_subjects']), \
"Cannot do both within and between subjects for a single person environment"

assert not (outlier_selection['within_group'] and outlier_selection['between_group_and_individual']), \
"Cannot do both within group and between group and subject for a multiple person environment"

In [ ]:
def sliding_window_creation():
    """Create sliding windows based on parameters in setup block above."""
    #Load data
    subject_data = []
    for filename in subjects:
        with open(filename, 'rb') as file:
            unpickled = pickle.load(file)
            if len(unpickled.index) < 100*(window['size'] - window['stride']):
                raise Exception("subject must have at least enough data for 100 windows, can only make " + \
                                f"{len(unpickled.index)/(window['size'] - window['stride'])} for subject {filename}.")
            unpickled = filter_data(unpickled, filename)
            subject_data.append(unpickled)
    
    #Create windows of data
    windowed_subject_data = []
    for subject in subject_data:
        windowed_subject_data.append(create_windows(subject))
    
    #Create mix data and add to windowed subject data
    if mixing > 0:
        #create set of files not yet used
        potential_mix_subjects = list(set([subjects_path + x for x in subject_directory_files]) - set(subjects))
        #pick a random one
        mix_subject = random.choice(potential_mix_subjects)
        #extract data and window it
        with open(mix_subject, 'rb') as file:
            unpickled = pickle.load(file)
            unpickled = filter_data(unpickled, "MIX: " + mix_subject)
            mix_data_window = create_windows(unpickled)
            mix_data = []
            #create random mix sample for every subject data set based on desired fraction 
            #or entire mix_data if |mix_data| < mixing*|windowed_subject|
            for subject in windowed_subject_data:
                mix_data.append(mix_data_window.sample(n=min(len(mix_data_window.values), \
                                                              int(mixing*len(subject.values)))))
    
    if combine_subject_dataframes:
        return pd.concat(windowed_subject_data, ignore_index=True), mix_data
    else:
        return windowed_subject_data, mix_data
    
def normalize(window):
    mean = window.mean(skipna=True)
    std = window.std(ddof=0, skipna=True)
    window = (window - mean)/std
    return window

def filter_data(data, filename):
    #Interpolate missing data
    data['hr'] = data['hr'].ffill().bfill()
    data['steps'] = data['steps'].ffill().bfill()
    data['steps'] = data['steps'].sparse.to_dense()
    
    #Record mean & std
    print(f"{filename} - Mean hr: {data['hr'].mean(skipna=True)} - STD hr: {data['hr'].std(ddof=0, skipna=True)}")
    print(f"{filename} - Mean steps: {data['steps'].mean(skipna=True)} - STD steps: {data['steps'].std(ddof=0, skipna=True)}")
    
    #Normalize if needed
    if data_sel['norm_hr']:
        data['hr'] = normalize(data['hr'])
    if data_sel['norm_steps']:
        data['steps'] = normalize(data['steps'])
        
    data["elapsed_time"] = (data["time"] - data["time"][0]).dt.total_seconds()
    
    #Select desired data
    if data_sel['hr'] and data_sel['steps']:
        return data
    elif data_sel['hr']:
        return data.drop(['steps'], axis=1)
    elif data_sel['steps']:
        return data.drop(['hr'], axis=1)
    else:
        raise ValueError("Can't make windows with no data, dummy")

def describe(window, start_time, end_time, prefix = ""):
    description = {}
    if features['count']:
        description[prefix + 'count'] = len(window)
    if features['mean']:
        description[prefix + 'mean'] = window.mean(skipna=True)
    if features['std']:
        description[prefix + 'std'] = window.std(ddof=0, skipna=True)
    if features['min']:
        description[prefix + 'min'] = window.min()
    if len(features['percentiles']) > 0:
        for quant in features['percentiles']:
            description[prefix + str(quant)] = window.quantile(quant)
    if features['max']:
        description[prefix + 'max'] = window.max()
    if features['n_th_step']:
        description[prefix + 'n_th_step'] = sc.stats.moment(window, moment = n_th_step_moment, nan_policy='omit')
    if features['mfcc']:
        time = (end_time - start_time)
        sr = len(window.values)/time
        res = lib.feature.mfcc(y=window.to_numpy(), sr=sr, n_mfcc=1, n_mels=1)[0]
        description[prefix + 'mfcc'] = np.mean(res)

    return description

def create_windows(subject_data):
    index = 0
    data = []
    window_end_index = index + window['size']
    new_start_index = window_end_index - window['stride']
    while new_start_index < subject_data['elapsed_time'].values[-1]:
#         print(f"{new_start_index}, {subject_data['elapsed_time'].values[-1]}")
        #Create, describe, and store windows
        new_window = subject_data[subject_data['elapsed_time'].between(index, window_end_index, inclusive='left')]
        if len(new_window.values) == 0:
            index = new_start_index
            window_end_index = index + window['size']
            new_start_index = window_end_index - window['stride']
            continue
        new_row = {}
        if data_sel['hr'] and data_sel['steps']:
            hr_window = new_window['hr']
            new_row.update(describe(hr_window, index, window_end_index,"hr_"))
            steps_window = new_window['steps']
            new_row.update(describe(steps_window, index, window_end_index,"steps_"))
        elif data_sel['hr']:
            hr_window = new_window['hr']
            new_row.update(describe(hr_window, index, window_end_index,"hr_"))
        elif data_sel['steps']:
            steps_window = new_window['steps']
            new_row.update(describe(steps_window, index, window_end_index,"steps_"))
            
        data.append(pd.Series(new_row))
        
        #Prepare for next loop
        index = new_start_index
        window_end_index = index + window['size']
        new_start_index = window_end_index - window['stride']
        
    return pd.DataFrame(data, columns=new_row.keys())

def train_without_outliers(normal_data, outlier_data, addition_normal_to_test):
    normalizer = StandardScaler()
    normalizer.set_output(transform='pandas')
    
    #Create new test data and drop it from the original data
    additional_test_data = normal_data.sample(n=addition_normal_to_test)
    normal_data = pd.concat([normal_data, additional_test_data]).drop_duplicates(keep=False)
    
    #Add class indication, combine, and shuffle
    additional_test_data['class'] = 1
    outlier_data['class'] = 0
    test_data = pd.concat([additional_test_data, outlier_data])
    test_data = test_data.sample(frac=1).reset_index(drop=True)
    
    normal_data = normalizer.fit_transform(normal_data)
    test_classes = test_data['class']
    test_data = normalizer.transform(test_data.drop(['class'], axis=1))
    
    return normal_data, test_data, np.ones(len(normal_data)), test_classes
#     return normal_data, test_data.drop(['class'], axis=1), np.ones(len(normal_data.values)), test_data['class']

## Gaussian Mixture Model
### Single Person

In [ ]:
#GAUSSIAN SINGLE PERSON OVERRIDE VARIABLES.
#IF DESIRE TO USE, RUN THIS BEFORE THE BELOW CELLS AND DO NOT RUN THE GENERAL ONE IN BETWEEN
subjects = random.choices(subject_directory_files, k = 1)
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]
combine_subject_dataframes = True

#Cutoff percentile at which scores are either classified normal or outlier points
base_cutoff = 25

#Up to which value to test best value for n_components, testing all integers in the range 1,n_limit
n_limit = 15

In [ ]:
def train_test_validate_split(df, mix_data):
    if not outlier_selection['between_subjects']:
        numpy_data = df.to_numpy()
        window_lengths = [np.linalg.norm(x) for x in numpy_data]
        outlier_indexes = np.argpartition(window_lengths, (-1 * int(0.1*len(numpy_data))))[(-1 * int(0.1*len(numpy_data))):]
        outliers = df.iloc[outlier_indexes]
        data_without_outliers = pd.concat([df, outliers]).drop_duplicates(keep=False)
    else:
        outliers = mix_data[0]
        data_without_outliers = df
    
    #train test split
    X_train, X_test, y_train, y_test = train_without_outliers(data_without_outliers, outliers, int(len(df)*0.2))
    
    #validation test split
    X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
    
    if outlier_selection['within_subject']:
        X_train = pd.concat([X_train, mix_data[0]], ignore_index=True)
        y_train = np.ones(len(X_train.values))
    
    return X_train, X_validate, X_test, y_train, y_validate, y_test

def gmm_aic_score(estimator, X):
    """Callable to pass to GridSearchCV that will use the AIC score."""
    # Make it negative since GridSearchCV expects a score to maximize
    return -estimator.aic(X)

def find_best_params(X_validate):
    param_grid = {
        "n_components": range(1, n_limit),
        "covariance_type": ["spherical", "tied", "diag", "full"],
    }
    grid_search = GridSearchCV(
        GaussianMixture(), param_grid=param_grid, scoring=gmm_aic_score
    )
    grid_search.fit(X_validate)

    return grid_search.best_params_['covariance_type'], grid_search.best_params_['n_components']

def roc_curving(cov_type, n_com, X_train, X_validate, y_train, y_validate, print_curve=True):
    #Fit on GMM and determine scores
    gmm = GaussianMixture(covariance_type = cov_type, n_components = n_com)
    gmm.fit(X_train.to_numpy())
    scores = gmm.score_samples(X_validate.to_numpy())
    
    #Get ROC and AUC
    auc = roc_auc_score(y_validate.to_numpy(), np.array(scores))
    fpr, tpr, thresholds_sk = roc_curve(y_validate.to_numpy(), np.array(scores))
    if print_curve:
        plt.title('Receiver Operating Characteristic')
        plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % auc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()

    optimal_score_cutoff = sorted(list(zip(np.abs(np.subtract(tpr, fpr)), thresholds_sk)), key=lambda i: i[0], reverse=True)[0][1]
    return optimal_score_cutoff, auc

def prep_and_test(cov_type, n_com, X_train, X_test, y_train, y_test, thres = base_cutoff):
    #Train and determine outliers
    gmm = GaussianMixture(covariance_type = cov_type, n_components = n_com)
    gmm.fit(X_train.to_numpy())
    scores = gmm.score_samples(X_test.to_numpy())
#     pct_threshold = np.percentile(scores, thres)
    predictions = pd.Series(scores).apply(lambda x: 0 if x < thres else 1)
    TP = [1 for x, y in zip(predictions, y_test.tolist()) if x == 0 and y == 0]
    FP = [1 for x, y in zip(predictions, y_test.tolist()) if x == 0 and y == 1]
    TN = [1 for x, y in zip(predictions, y_test.tolist()) if x == 1 and y == 1]
    FN = [1 for x, y in zip(predictions, y_test.tolist()) if x == 1 and y == 0]
    return len(TP), len(FP), len(TN), len(FN)

In [ ]:
subject_window, mix_data = sliding_window_creation()

printing_section = subject_window.loc[0:10].copy()
display(printing_section.style.set_caption("Initial 10 windows of subject"))
print(f"Number of rows: {len(subject_window.values)}")

In [ ]:
X_train, X_validate, X_test, y_train, y_validate, y_test = train_test_validate_split(subject_window, mix_data)

In [ ]:
cov_type, n_com = find_best_params(X_validate)
print(f"Found best covariance type: {cov_type} and number of components: {n_com}")

In [ ]:
optimal_cutoff = roc_curving(cov_type, n_com, X_train, X_validate, y_train, y_validate)
# print(f"Default cutoff provided: {base_cutoff}")
print(f"Cutoff from ROC curve: {optimal_cutoff}")

# TP, FP, TN, FN = prep_and_test(X_train, X_test, y_train, y_test)
# default_acc = (TP + TN) / (TP + FP + TN + FN)

TP, FP, TN, FN = prep_and_test(cov_type, n_com, X_train, X_test, y_train, y_test, optimal_cutoff)
roc_acc = (TP + TN) / (TP + FP + TN + FN)

# print(f"Default accuracy: {default_acc}")
print(f"ROC accuracy: {roc_acc}")

### Multiple Persons

In [ ]:
#GAUSSIAN MULTIPLE PERSONS OVERRIDE VARIABLES.
#IF DESIRE TO USE, RUN THIS BEFORE THE BELOW CELLS AND DO NOT RUN THE GENERAL ONE IN BETWEEN

subjects = random.sample(subject_directory_files, 6)
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]

combine_subject_dataframes = False

#Cutoff percentile at which scores are either classified normal or outlier points
base_cutoff = 33

#Up to which value to test best value for n_components, testing all integers in the range 1,n_limit
n_limit = 15

In [ ]:
def prep_train_test_data(dfs, mix_data):
    #Prep DataFrames for the GaussianMixture function and determine outliers
    if not outlier_selection['between_group_and_individual']:
        numpy_frames = [df.to_numpy() for df in dfs]
        all_outliers = []
        all_normal_data = []
        for df, numpy_data in zip(dfs, numpy_frames):
            window_lengths = [np.linalg.norm(x) for x in numpy_data]
            outlier_indexes = np.argpartition(window_lengths, (-1 * int(0.1*len(numpy_data))))[(-1 * int(0.1*len(numpy_data))):]
            outliers = df.iloc[outlier_indexes]
            all_outliers.append(outliers)
            data_without_outliers = pd.concat([df, outliers]).drop_duplicates(keep=False)
            all_normal_data.append(data_without_outliers)
    else:
        all_outliers = [pd.concat(mix_data, ignore_index=True)]
        all_normal_data = [pd.concat(dfs, ignore_index=True)]
    
    #Create train batches from x number of normal windows of every subject, and a test set for testing
    x_train_data = []
    x_test_data = []
    x_validate_data = []
    y_validate_data = []
    y_train_data = []
    y_test_data = []
    for data_without_outliers, outliers in zip(all_normal_data, all_outliers):
        X_train, X_test, y_train, y_test = train_without_outliers(data_without_outliers, outliers, int(len(outliers)*2))
        
        #validation test split
        X_validate, X_test, y_validate, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
        
        x_train_data.append(X_train.values.tolist())
        x_test_data.append(X_test.values.tolist())
        x_validate_data.append(X_validate.values.tolist())
        y_validate_data.append(y_validate.tolist())
        y_train_data.append(y_train)
        y_test_data.append(y_test.tolist())
    
    if outlier_selection['within_group']:
        X_train = pd.concat([X_train, outliers], ignore_index=True)
        y_train = np.ones(len(X_train.values))
    
    X_train = list(itertools.chain.from_iterable(x_train_data))
    X_test = list(itertools.chain.from_iterable(x_test_data))
    X_validate = list(itertools.chain.from_iterable(x_validate_data))
    y_validate = list(itertools.chain.from_iterable(y_validate_data))
    y_train = list(itertools.chain.from_iterable(y_train_data))
    y_test = list(itertools.chain.from_iterable(y_test_data))
    
    return np.array(X_train), np.array(X_validate), np.array(X_test), y_train, y_validate, y_test


def roc_curve_multiple(cov_type, n_com, X_train, X_validate, y_train, y_validate, print_curve=True):
    gmm = GaussianMixture(covariance_type = cov_type, n_components = n_com)
    gmm.fit(X_train)
    scores = gmm.score_samples(X_validate)
    
    #Get ROC and AUC
    auc = roc_auc_score(y_validate, np.array(scores))
    fpr, tpr, thresholds_sk = roc_curve(y_validate, np.array(scores))
    if print_curve:
        plt.title('Receiver Operating Characteristic')
        plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % auc)
        plt.legend(loc = 'lower right')
        plt.plot([0, 1], [0, 1],'r--')
        plt.xlim([0, 1])
        plt.ylim([0, 1])
        plt.ylabel('True Positive Rate')
        plt.xlabel('False Positive Rate')
        plt.show()

    optimal_score_cutoff = sorted(list(zip(np.abs(np.subtract(tpr, fpr)), thresholds_sk)), key=lambda i: i[0], reverse=True)[0][1]
    return optimal_score_cutoff


def train_and_test_multiple(cov_type, n_com, X_train, X_test, y_train, y_test, thres = base_cutoff):   
    #Train and determine outliers
    gmm = GaussianMixture(covariance_type = cov_type, n_components = n_com)
    gmm.fit(X_train)
    scores = gmm.score_samples(X_test)
#     pct_threshold = np.percentile(scores, thres)
    predictions = pd.Series(scores).apply(lambda x: 0 if x < thres else 1)
    TP = [1 for x, y in zip(predictions, y_test) if x == 0 and y == 0]
    FP = [1 for x, y in zip(predictions, y_test) if x == 0 and y == 1]
    TN = [1 for x, y in zip(predictions, y_test) if x == 1 and y == 1]
    FN = [1 for x, y in zip(predictions, y_test) if x == 1 and y == 0]
    return len(TP), len(FP), len(TN), len(FN)

In [ ]:
subjects_window, mix_data = sliding_window_creation()
print("Preprocessed data")

In [ ]:
X_train, X_validate, X_test, y_train, y_validate, y_test = prep_train_test_data(subjects_window, mix_data)
print("Prepped train and test data")

In [ ]:
cov_type, n_com = find_best_params(X_validate)
print(f"Found best covariance type: {cov_type} and number of components: {n_com}")

In [ ]:
optimal_cutoff = roc_curve_multiple(cov_type, n_com, X_train, X_validate, y_train, y_validate)
# print(f"Default cutoff provided: {base_cutoff}")
print(f"Cutoff from ROC curve: {optimal_cutoff}")

In [ ]:
# TP, FP, TN, FN = train_and_test_multiple(X_train, X_test, y_train, y_test)
# default_acc = (TP + TN) / (TP + FP + TN + FN)

TP, FP, TN, FN = train_and_test_multiple(cov_type, n_com, X_train, X_test, y_train, y_test, optimal_cutoff)
roc_acc = (TP + TN) / (TP + FP + TN + FN)

# print(f"Default outlier accuracy: {default_acc}")
print(f"ROC outlier accuracy: {roc_acc}")

## DBSCAN
### Single Person

In [ ]:
#DBSCAN SINGLE OVERRIDE VARIABLES.
#IF DESIRE TO USE, RUN THIS BEFORE THE BELOW CELLS AND DO NOT RUN THE GENERAL ONE IN BETWEEN

subjects = random.choices(subject_directory_files, k = 1)
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]
combine_subject_dataframes = True

feature_count = 0
for value in features.values():
    if type(value) is bool:
        if value:
            feature_count = feature_count + 1
    if type(value) is list:
        feature_count = feature_count + len(value)
if data_sel['hr'] and data_sel['steps']:
    feature_count = feature_count * 2

minPts = 2*feature_count #Sander et al., 1998


In [ ]:
df, mixing_data = sliding_window_creation()
mixing_data = mixing_data[0]
y_true=[]
if outlier_selection['within_subject']:
    numpy_data = df.to_numpy()
    window_lengths = [np.linalg.norm(x) for x in numpy_data]
    outlier_indexes = np.argpartition(window_lengths, (-1 * int(0.1*len(numpy_data))))[(-1 * int(0.1*len(numpy_data))):]
    y_true = np.ones(len(numpy_data))
    y_true[outlier_indexes] = 0
    df['class'] = y_true
    mixing_data['class'] = np.ones(len(mixing_data.values))
    df = pd.concat([df,mixing_data], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True) #shuffle df
    y_true = df['class'].tolist()
    df = df.drop(['class'], axis=1)
else:
    df['class'] = np.ones(len(df.values))
    mixing_data['class'] = np.zeros(len(mixing_data.values))
    df = pd.concat([df,mixing_data], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True) #shuffle df
    y_true = df['class'].tolist()
    df = df.drop(['class'], axis=1)


In [ ]:
def find_epsilon(df, minPts):
    """Find best epsilon using the elbow curve of K-NN distances and its point of maximum curviture"""
    neighbors = NearestNeighbors(n_neighbors=minPts)
    neighbors_fit = neighbors.fit(df)
    distances, indices = neighbors_fit.kneighbors(df)

    distances = np.sort(distances, axis=0)
    distances = distances[:,1]

    coords = np.column_stack((range(0,len(distances)), distances))
    rotor = Rotor()
    rotor.fit_rotate(coords)
    elbow_index = rotor.get_elbow_index()
    epsilon = coords[elbow_index][1]

    return epsilon
        

In [ ]:
epsilon = find_epsilon(df, minPts)

dbscan=DBSCAN(eps=epsilon, min_samples=minPts)
predictions = dbscan.fit_predict(df)

In [ ]:
cleaned_predictions = [0 if x == -1 else 1 for x in predictions]
TP = [1 for x, y in zip(cleaned_predictions, y_true) if x == 0 and y == 0]
FP = [1 for x, y in zip(cleaned_predictions, y_true) if x == 0 and y == 1]
TN = [1 for x, y in zip(cleaned_predictions, y_true) if x == 1 and y == 1]
FN = [1 for x, y in zip(cleaned_predictions, y_true) if x == 1 and y == 0]
print(epsilon)
acc = (len(TP) + len(TN)) / (len(TP) + len(FP) + len(TN) + len(FN))
print(f"Accuracy: {acc}")

from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(y_true, cleaned_predictions)
print(f"Adjusted Rand Index: {ari}")

from sklearn.metrics import rand_score
ari = rand_score(y_true, cleaned_predictions)
print(f"Rand Index: {ari}")

from sklearn import metrics
ss = metrics.silhouette_score(df, y_true)
print(f"Silhouette Score: {ss}")

### Multiple Persons

In [ ]:
#DBSCAN MULTIPLE PERSONS OVERRIDE VARIABLES.
#IF DESIRE TO USE, RUN THIS BEFORE THE BELOW CELLS AND DO NOT RUN THE GENERAL ONE IN BETWEEN

subjects = random.sample(subject_directory_files, 6)
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]

combine_subject_dataframes = False

feature_count = 0
for value in features.values():
    if type(value) is bool:
        if value:
            feature_count = feature_count + 1
    if type(value) is list:
        feature_count = feature_count + len(value)
if data_sel['hr'] and data_sel['steps']:
    feature_count = feature_count * 2
    
minPts = 2*feature_count #Sander et al., 1998

In [ ]:
dfs, mixing_data = sliding_window_creation()

In [ ]:
y_true = []
df = None
if outlier_selection['within_group']:
    y_trues = []
    new_dfs = []
    for df in dfs:
        numpy_data = df.to_numpy()
        window_lengths = [np.linalg.norm(x) for x in numpy_data]
        outlier_indexes = np.argpartition(window_lengths, (-1 * int(0.1*len(numpy_data))))[(-1 * int(0.1*len(numpy_data))):]
        y_true = np.zeros(len(numpy_data))
        y_true[outlier_indexes] = 1
        df['class'] = y_true
        new_dfs.append(df)

    df = pd.concat(new_dfs, ignore_index=True)
    y_true = df['class'].tolist()
    df = df.drop(['class'], axis=1)
else:
    df = pd.concat(dfs, ignore_index=True)
    df['class'] = np.ones(len(df.values))
    mix = pd.concat(mixing_data, ignore_index=True)
    mix['class'] = np.zeros(len(mix.values))
    df = pd.concat([df,mix], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True) #shuffle df
    y_true = df['class'].tolist()
    df = df.drop(['class'], axis=1)

In [ ]:
def find_epsilon(df, minPts):
    """Find best epsilon using the elbow curve of K-NN distances and its point of maximum curviture"""
#     results = []
#     for df in dfs:
    neighbors = NearestNeighbors(n_neighbors=minPts)
    neighbors_fit = neighbors.fit(df)
    distances, indices = neighbors_fit.kneighbors(df)

    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    if outlier_selection['within_group']:
        distances = (distances - np.mean(distances))/np.std(distances)

    coords = np.column_stack((range(0,len(distances)), distances))
    rotor = Rotor()
    rotor.fit_rotate(coords)
    elbow_index = rotor.get_elbow_index()
    epsilon = coords[elbow_index][1]
#         results.append(epsilon)
#     print(results)
    return epsilon
#     return np.median(results)

In [ ]:
epsilon = find_epsilon(df, minPts)
print(f"epsilon: {epsilon}")
dbscan=DBSCAN(eps=epsilon, min_samples=minPts)
predictions = dbscan.fit_predict(df)

In [ ]:
cleaned_predictions = [0 if x == -1 else 1 for x in predictions]
TP = [1 for x, y in zip(cleaned_predictions, y_true) if x == 0 and y == 0]
FP = [1 for x, y in zip(cleaned_predictions, y_true) if x == 0 and y == 1]
TN = [1 for x, y in zip(cleaned_predictions, y_true) if x == 1 and y == 1]
FN = [1 for x, y in zip(cleaned_predictions, y_true) if x == 1 and y == 0]

acc = (len(TP) + len(TN)) / (len(TP) + len(FP) + len(TN) + len(FN))
print(f"Accuracy: {acc}")

from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(y_true, cleaned_predictions)
print(f"Adjusted Rand Index: {ari}")

from sklearn.metrics import rand_score
ari = rand_score(y_true, cleaned_predictions)
print(f"Rand Index: {ari}")

from sklearn import metrics
ss = metrics.silhouette_score(df, y_true)
print(f"Silhouette Score: {ss}")

## Bayesian Outlier Detection
### Single Person

In [ ]:
#BOD SINGLE PERSON OVERRIDE VARIABLES.
#IF DESIRE TO USE, RUN THIS BEFORE THE BELOW CELLS AND DO NOT RUN THE GENERAL ONE IN BETWEEN

column_name = 'window_length'
prior_mean = None
prior_std = None

threshold = 0.2

subjects = random.choices(subject_directory_files, k = 1)
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]

In [ ]:
def bayesian_outlier(data_frame):
    column_data = data_frame[column_name].values
    
    # Compute the sample mean and sample standard deviation
    sample_mean = np.mean(column_data)
    sample_std = np.std(column_data)
    
    # Compute the posterior mean and posterior standard deviation
    posterior_precision = 1.0 / (1.0 / prior_std**2 + len(column_data) / sample_std**2)
    posterior_mean = posterior_precision * (prior_mean / prior_std**2 + np.sum(column_data) / sample_std**2)
    posterior_std = np.sqrt(1.0 / posterior_precision)
    
    # Compute the outlier probability for each data point
    outlier_probs = np.exp(-0.5 * ((column_data - posterior_mean) / posterior_std)**2)
    
    # Determine the outliers based on the threshold
    outliers = data_frame[outlier_probs < threshold]
    
    return outliers

In [ ]:
subject_windows = sliding_window_creation()

In [ ]:
numpy_data = subject_windows.to_numpy()
window_lengths = [np.linalg.norm(x) for x in numpy_data]
subject_windows['window_length'] = pd.Series(window_lengths)
outlier_indexes = np.argpartition(window_lengths, (-1 * int(0.1*len(numpy_data))))[(-1 * int(0.1*len(numpy_data))):]
outliers = subject_windows.iloc[outlier_indexes]
data_without_outliers = pd.concat([subject_windows, outliers]).drop_duplicates(keep=False)

In [ ]:
prior_mean = np.mean(data_without_outliers['window_length'])
prior_std = np.std(data_without_outliers['window_length'])

predicted_outliers = bayesian_outlier(subject_windows)

In [ ]:
outliers_np = outliers.to_numpy()
negative_np = data_without_outliers.to_numpy()
predict_np = predicted_outliers.to_numpy()
neg_predict_np = (pd.concat([subject_windows, predicted_outliers]).drop_duplicates(keep=False)).to_numpy()
TP = [1 for x in predict_np if any(np.equal(outliers_np,x).all(1))]
FP = [1 for x in predict_np if not any(np.equal(outliers_np,x).all(1))]
TN = [1 for x in neg_predict_np if any(np.equal(negative_np,x).all(1))]
FN = [1 for x in neg_predict_np if not any(np.equal(negative_np,x).all(1))]        

acc = (len(TP) + len(TN)) / (len(TP) + len(FP) + len(TN) + len(FN))
print(f"Accuracy: {acc}")

### Multiple Persons

In [ ]:
#BOD MULTIPLE PERSONS OVERRIDE VARIABLES.
#IF DESIRE TO USE, RUN THIS BEFORE THE BELOW CELLS AND DO NOT RUN THE GENERAL ONE IN BETWEEN

column_name = 'window_length'
prior_mean = None
prior_std = None

threshold = 0.2

subjects = random.sample(subject_directory_files, 6)
subjects = [(subjects_path + x) if x.endswith('.pkl') else (subjects_path + (x + '.pkl')) for x in subjects]

In [ ]:
def bayesian_outlier(data_frame):
    column_data = data_frame[column_name].values
    
    # Compute the sample mean and sample standard deviation
    sample_mean = np.mean(column_data)
    sample_std = np.std(column_data)
    
    # Compute the posterior mean and posterior standard deviation
    posterior_precision = 1.0 / (1.0 / prior_std**2 + len(column_data) / sample_std**2)
    posterior_mean = posterior_precision * (prior_mean / prior_std**2 + np.sum(column_data) / sample_std**2)
    posterior_std = np.sqrt(1.0 / posterior_precision)
    
    # Compute the outlier probability for each data point
    outlier_probs = np.exp(-0.5 * ((column_data - posterior_mean) / posterior_std)**2)
    
    # Determine the outliers based on the threshold
    outliers = data_frame[outlier_probs < threshold]
    
    return outliers

In [ ]:
subject_windows = sliding_window_creation()

In [ ]:
numpy_data = subject_windows.to_numpy()
window_lengths = [np.linalg.norm(x) for x in numpy_data]
subject_windows['window_length'] = pd.Series(window_lengths)
outlier_indexes = np.argpartition(window_lengths, (-1 * int(0.1*len(numpy_data))))[(-1 * int(0.1*len(numpy_data))):]
outliers = subject_windows.iloc[outlier_indexes]
data_without_outliers = pd.concat([subject_windows, outliers]).drop_duplicates(keep=False)

In [ ]:
prior_mean = np.mean(data_without_outliers['window_length'])
prior_std = np.std(data_without_outliers['window_length'])

predicted_outliers = bayesian_outlier(subject_windows)

In [ ]:
outliers_np = outliers.to_numpy()
negative_np = data_without_outliers.to_numpy()
predict_np = predicted_outliers.to_numpy()
neg_predict_np = (pd.concat([subject_windows, predicted_outliers]).drop_duplicates(keep=False)).to_numpy()
TP = [1 for x in predict_np if any(np.equal(outliers_np,x).all(1))]
FP = [1 for x in predict_np if not any(np.equal(outliers_np,x).all(1))]
TN = [1 for x in neg_predict_np if any(np.equal(negative_np,x).all(1))]
FN = [1 for x in neg_predict_np if not any(np.equal(negative_np,x).all(1))]        

acc = (len(TP) + len(TN)) / (len(TP) + len(FP) + len(TN) + len(FN))
print(f"Accuracy: {acc}")